# Проект для Интернет-магазина c BERT

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Мы обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import notebook


from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sklearn.svm

import optuna

import warnings
warnings.filterwarnings("ignore")

### Создадим токенайзер и модель

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Прочитаем таблицу

In [3]:
df_tweets = pd.read_csv('toxic_comments.csv')
df_tweets.info()
df_tweets.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


В таблице 2 столбца. Один столбец *text* с текстами комментаторов, который нам предстоит перевести в признаки для обучения и предсказания нашей модели, а второй столбец *toxic* содержит целевой признак. Что бы хотя бы немного ускорить работу программы, переведём данные в столбце *toxic* из int64 в uint8 

In [4]:
df_tweets['toxic'] = df_tweets['toxic'].astype('uint8')
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  uint8 
dtypes: object(1), uint8(1)
memory usage: 1.4+ MB


Выборку уменьшим, что бы не ждать бескнечно долго рассчётов дальнейшего эмбеддинга.

In [5]:
df_tweets = df_tweets.sample(20000, replace=True).reset_index(drop=True)

In [6]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

### Произведём эмбеддинг

In [ ]:

batch_size = 400
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/50 [00:00<?, ?it/s]

### Разделим таблицу на выборки

Объединим эмбеддинги, чтобы получить признаки для дальнейшей разбивки на тренировочную и тестовую выборки, а также получим целевой признак из столбца `df_tweets['toxic']`.

In [ ]:
features = np.concatenate(embeddings)

target = df_tweets['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=12345)

## Обучим модели

### Модель логистической регрессии

In [ ]:
model_log = LogisticRegression()
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_test)
f1_log = f1_score(target_test, predictions)
print('F1 модели логистической регрессии:', '{:.2f}'.format(f1_log))

### Модель случайного леса

Для подбора гиперпараметров используем библиотеку *optuna*.

In [ ]:
def objective(trial):
    rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 20))
    rf_n_estimators = int(trial.suggest_loguniform('rf_n_estimators', 2, 80))
    classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators, random_state=12345)
        
    score = cross_val_score(classifier_obj, features, target, scoring='f1', n_jobs=-1, cv=5)
    f1_score = score.mean()
    return f1_score

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

Лучший результат с гиперпараметрами max_depth=16, rf_n_estimators=46.

In [ ]:
model_forest = RandomForestClassifier(max_depth=11, n_estimators=5, random_state=12345)
model_forest.fit(features_train, target_train)
predictions_forest = model_forest.predict(features_test)
f1_forest = f1_score(target_test, predictions_forest)
print('F1 модели случайного леса:', '{:.2f}'.format(f1_forest))

### Модель LightGBM

In [ ]:
model_gbm = LGBMClassifier(random_state=12345, class_weight='balanced')
model_gbm.fit(features_train, target_train, verbose=10)

predictions_gbm = model_gbm.predict(features_test)
f1_gbm = f1_score(target_test, predictions_gbm)

print('F1 модели градиентного бустинга библиотеки lightGBM:', '{:.2f}'.format(f1_gbm))

### Модель CatBoost

In [ ]:
model_cat = CatBoostClassifier(iterations=100, random_seed=12345)
model_cat.fit(features_train, target_train, verbose=10)

predictions_cat = model_cat.predict(features_test)
f1_cat = f1_score(target_test, predictions_cat)

print('F1 модели градиентного бустинга библиотеки CatBoost:', '{:.2f}'.format(f1_cat))

## Выводы

Для наглядного сравнения метрик моделей сделаем таблицу и график.

In [ ]:
final_table = pd.DataFrame(index=[
    'Линейная регрессия', 
    'Случайный лес', 
    'LGBM', 
    'CatBoost'
], columns=['F1'], data=[
    f1_log, 
    f1_forest, 
    f1_gbm, 
    f1_cat]
                          )

display(final_table)
final_table.plot.barh()
plt.ylabel('Модели')
plt.xlabel('F1')
plt.title('F1 моделей')
plt.show()

Лучшей моделью для получения предсказаний является модель логистической регрессии.